# Introduction to Semantic Kernel and Basic Agent Creation

Welcome to your first hands-on experience with AI agents! In this notebook, you'll learn how to use [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel), Microsoft's open-source AI orchestration framework, to create intelligent agents.

## What You'll Learn

By the end of this notebook, you'll understand:

1. **What Semantic Kernel is** - An AI framework that helps you integrate Large Language Models (LLMs) with your applications
2. **Core Components** - Agents, plugins, functions, and how they work together
3. **Basic Agent Creation** - How to build a simple AI agent that can interact with users
4. **Function Calling** - How agents can use custom tools (plugins) to perform specific tasks
5. **Conversation Management** - How to maintain context across multiple interactions

## What is Semantic Kernel?

Semantic Kernel is Microsoft's lightweight SDK that lets you mix conventional programming languages (like Python, C#, Java) with Large Language Models (LLMs). It provides:

- **Agent Framework**: Easy creation of AI agents with specific roles and capabilities
- **Plugin System**: Extensible architecture for adding custom functions and tools
- **Multi-Model Support**: Works with various AI models (Azure OpenAI, OpenAI, Hugging Face, etc.)
- **Memory Management**: Built-in conversation history and context management

## The Travel Agent Example

In this notebook, we'll build a **Travel Agent** - a simple but practical AI agent that can:
- Suggest vacation destinations
- Provide personalized travel recommendations
- Handle follow-up questions and requests
- Use custom functions to access destination data

This example demonstrates the fundamental patterns you'll use in more advanced agentic scenarios throughout this workshop.

## Import the Required Python Packages

Before we start building our agent, we need to import the necessary libraries. Each import serves a specific purpose in our agent architecture:

### Core Python Libraries
- `os` - For accessing environment variables (API keys, endpoints)
- `json` - For handling JSON data in function calls
- `typing.Annotated` - For type hints that improve function descriptions

### Azure Authentication
- `azure.identity.DefaultAzureCredential` - For Azure AD authentication (optional)
- `dotenv.load_dotenv` - For loading environment variables from .env files

### Jupyter Notebook Display
- `IPython.display` - For rich HTML output in our notebook

### Semantic Kernel Components
- `ChatCompletionAgent` - The main agent class that handles conversations
- `ChatHistoryAgentThread` - Manages conversation history and context
- `AzureChatCompletion` - Connector for Azure OpenAI services
- `FunctionCallContent`, `FunctionResultContent`, `StreamingTextContent` - Content types for handling different response formats
- `kernel_function` - Decorator for creating custom agent functions (tools)

In [6]:
import os 
import json
from typing import Annotated
from azure.identity import DefaultAzureCredential

from dotenv import load_dotenv

from IPython.display import display, HTML

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

## Creating the AI Service Connection

To build an AI agent, we first need to establish a connection to a Large Language Model (LLM). In this example, we'll use [Azure OpenAI](https://azure.microsoft.com/en-us/products/ai-services/openai-service).

### Why Azure OpenAI?

Azure OpenAI provides:
- **Enterprise-grade security** - Your data stays within your Azure tenant
- **Reliable performance** - SLA-backed service with consistent availability
- **Compliance** - Meets enterprise compliance requirements
- **Integration** - Seamless integration with other Azure services

### Configuration Explained

The `AzureChatCompletion` connector requires several parameters:

- **`deployment_name`** - The name of your model deployment (e.g., "gpt-4o-mini")
- **`endpoint`** - Your Azure OpenAI service endpoint URL
- **`api_version`** - The API version to use (we use "2024-02-01" for latest features)
- **`api_key`** - Your service authentication key

### Authentication Options

We provide two authentication methods:

1. **API Key Authentication** (Recommended for development)
   - Simple and straightforward
   - Uses a service-specific API key
   - Good for local development and testing

2. **Azure AD Authentication** (Recommended for production)
   - Uses your Azure identity
   - Follows principle of least privilege
   - Better for production environments and CI/CD pipelines

💡 **Tip**: Store your credentials in environment variables or Azure Key Vault, never hard-code them in your source code!

In [7]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

## Understanding Plugins and Functions

Before we create our agent, let's understand one of Semantic Kernel's most powerful features: **Plugins**.

### What are Plugins?

Plugins are collections of functions that extend your agent's capabilities. Think of them as "tools" that your agent can use to perform specific tasks. Without plugins, AI agents can only generate text. With plugins, they can:

- Call external APIs
- Access databases
- Perform calculations
- Interact with other services
- Execute custom business logic

### The `@kernel_function` Decorator

The `@kernel_function` decorator transforms a regular Python method into an agent-callable function. Key features:

- **Auto-discovery**: The agent automatically knows about these functions
- **Type Safety**: Uses Python type hints for parameter validation
- **Documentation**: The `description` parameter helps the agent understand when to call the function
- **Return Types**: `Annotated` types provide clear descriptions of what the function returns

### Our Example: DestinationsPlugin

We'll create a simple plugin that provides vacation destinations. This demonstrates:
- How to structure a plugin class
- How to implement kernel functions
- How to manage state within a plugin
- How to provide meaningful descriptions for the AI agent

In [8]:
load_dotenv()

# Option 1: Using API Key (recommended for development)
chat_completion_service = AzureChatCompletion(
    deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini"),
    endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-01"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY")
)

# Option 2: Using Azure AD Authentication (uncomment to use)
# Create Azure credential 
credential = DefaultAzureCredential()

# Create a token provider function
def get_azure_ad_token():
    """Function to get Azure AD token for OpenAI."""
    token = credential.get_token("https://cognitiveservices.azure.com/.default")
    return token.token

# chat_completion_service = AzureChatCompletion(
#     deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini"),
#     endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
#     api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-01"),
#     ad_token=get_azure_ad_token()
# )

## Creating the AI Agent

Now we'll create our **Travel Agent** using the `ChatCompletionAgent` class. This is where we bring together all the components: the AI service, the plugins, and the agent's personality.

### Agent Components Explained

A Semantic Kernel agent consists of:

1. **Service**: The AI model connection (our Azure OpenAI service)
2. **Plugins**: The tools/functions the agent can use
3. **Name**: An identifier for the agent (useful in multi-agent scenarios)
4. **Instructions**: The agent's "personality" and behavioral guidelines

### Crafting Effective Instructions

The `instructions` parameter is crucial - it defines:

- **Role and Purpose**: What the agent is supposed to do
- **Behavior Guidelines**: How it should interact with users
- **Decision Logic**: When to use specific functions or approaches
- **Conversation Flow**: How to structure interactions

### Key Instruction Patterns

Our Travel Agent instructions demonstrate several best practices:

- **Clear Role Definition**: "You are a helpful AI Agent that can help plan vacations"
- **Conditional Logic**: "When users specify a destination, always plan for that location"
- **Structured Responses**: Providing numbered options and clear guidance
- **User Preference Priority**: "Always prioritize user preferences"

💡 **Pro Tip**: Well-crafted instructions are often more important than having many plugins. They guide the agent's decision-making and ensure consistent, helpful behavior.

In [9]:
AGENT_INSTRUCTIONS = """You are a helpful AI Agent that can help plan vacations for customers.

Important: When users specify a destination, always plan for that location. Only suggest random destinations when the user hasn't specified a preference.

When the conversation begins, introduce yourself with this message:
"Hello! I'm your TravelAgent assistant. I can help plan vacations and suggest interesting destinations for you. Here are some things you can ask me:
1. Plan a day trip to a specific location
2. Suggest a random vacation destination
3. Find destinations with specific features (beaches, mountains, historical sites, etc.)
4. Plan an alternative trip if you don't like my first suggestion

What kind of trip would you like me to help you plan today?"

Always prioritize user preferences. If they mention a specific destination like "Bali" or "Paris," focus your planning on that location rather than suggesting alternatives.
"""

agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions=AGENT_INSTRUCTIONS,
)

# Other agent types available in Semantic Kernel:

# 1. OpenAI Assistant Agent - Uses OpenAI's Assistant API
# from semantic_kernel.agents.open_ai import OpenAIAssistantAgent
# assistant_agent = OpenAIAssistantAgent(
#     kernel=kernel,
#     service=openai_service,
#     name="AssistantAgent",
#     instructions="You are a helpful assistant."
# )

# 2. Aggregate Chat Agent - Coordinates multiple agents
# from semantic_kernel.agents import AggregateChatAgent
# aggregate_agent = AggregateChatAgent(
#     agents=[agent1, agent2, agent3],
#     selection_strategy=selection_strategy
# )

# 3. Group Chat Manager Agent - Manages group conversations
# from semantic_kernel.agents import GroupChatManager
# group_manager = GroupChatManager(
#     agents=[agent1, agent2],
#     max_turns=10,
#     termination_strategy=termination_strategy
# )

## Running the Agent

This is where the magic happens! We'll execute our agent and observe how it handles conversations, makes decisions, and uses its tools.

### Understanding Agent Execution

Our execution code demonstrates several important concepts:

#### 1. **Conversation Threading**
- `ChatHistoryAgentThread` maintains conversation context
- Each message builds on previous interactions
- The agent remembers what was discussed earlier

#### 2. **Streaming Responses**
- `invoke_stream()` provides real-time response streaming
- Users see the agent "thinking" and responding progressively
- Better user experience for longer responses

#### 3. **Function Call Handling**
The agent's response can contain different types of content:

- **`FunctionCallContent`**: When the agent decides to use a plugin function
- **`FunctionResultContent`**: The result returned from the function call
- **`StreamingTextContent`**: The agent's conversational response text

#### 4. **Rich Display Format**
Our code creates an HTML display that shows:
- User inputs clearly marked
- Function calls in expandable sections (to reduce clutter)
- Agent responses formatted nicely
- Clear conversation flow

### Test Scenarios

We've included two test inputs that demonstrate different agent behaviors:

1. **"Plan me a day trip"** - Tests the agent's ability to use the random destination function
2. **"I don't like that destination. Plan me another vacation"** - Tests conversational memory and follow-up handling

### What to Watch For

As you run this code, observe:
- How the agent decides when to call the `get_random_destination` function
- How it maintains context between messages
- How it handles the user's rejection and provides alternatives
- The natural conversation flow despite using structured function calls

💡 **Experiment**: Try modifying the `user_inputs` to test different scenarios and see how the agent responds!

In [12]:
# Test inputs to demonstrate agent capabilities
user_inputs = [
    "Plan me a day trip.",
    "I don't like that destination. Plan me another vacation.",
]

async def main():
    # Initialize conversation thread (maintains context across messages)
    thread: ChatHistoryAgentThread | None = None

    # Process each user input sequentially
    for user_input in user_inputs:
        # Start building HTML output for display
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        # Initialize response tracking variables
        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer for streaming function calls
        current_function_name = None
        argument_buffer = ""

        # Stream agent response in real-time
        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            # Update thread with conversation history
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            # Process each content item in the response
            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    # Capture function name when agent calls a tool
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate function arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Process completed function call
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            # Pretty-print JSON arguments if possible
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # Keep as raw string if not valid JSON

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    # Capture function result
                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    # Collect agent's text response
                    full_response.append(item.text)

        # Add collapsible function calls section to output
        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        # Add agent response to output
        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        # Display the formatted conversation
        display(HTML(html_output))

# Run the agent conversation
await main()

## 🎉 Congratulations!

You've just created your first AI agent using Semantic Kernel! Let's recap what you've accomplished:

### What You Built

✅ **AI Service Connection** - Connected to Azure OpenAI with proper authentication  
✅ **Custom Plugin** - Created a `DestinationsPlugin` with a `get_random_destination` function  
✅ **Intelligent Agent** - Built a Travel Agent with specific instructions and behavior  
✅ **Conversation Flow** - Implemented streaming responses with conversation memory  
✅ **Function Integration** - Watched the agent automatically decide when to use tools  

### Key Concepts Mastered

1. **Agent Architecture**: Service + Plugins + Instructions = Intelligent Agent
2. **Function Calling**: How agents use tools to extend their capabilities
3. **Conversation Management**: Maintaining context across multiple interactions
4. **Streaming Responses**: Real-time, progressive response generation

### Next Steps in Your AI Agent Journey

This foundation prepares you for more advanced patterns in the upcoming notebooks:

- **Advanced Tool Use** - More complex plugins and function orchestration
- **Agentic RAG** - Combining retrieval-augmented generation with agents
- **Multi-Agent Systems** - Coordinating multiple specialized agents
- **Planning & Design** - Agents that can break down complex tasks
- **Metacognition** - Self-aware agents that can reflect on their own processes

### Experiment and Explore

Try these modifications to deepen your understanding:

1. **Add new destinations** to the `destinations` list
2. **Create additional functions** in the plugin (e.g., `get_weather_for_destination`)
3. **Modify the agent instructions** to change its personality
4. **Add new test scenarios** in `user_inputs`
5. **Experiment with different models** by changing the deployment name

Ready for more advanced agent patterns? Continue to the next notebook! 🚀